# Домашнее задание

In [1]:
# !pip install opencv-contrib-python

In [2]:
import sys
import numpy as np
import cv2 as cv

In [3]:
def show_my_image(img, title=None):
    #cv.namedWindow(title, cv.WINDOW_NORMAL)
    cv.imshow(title, img)
    cv.waitKey()
    cv.destroyAllWindows()
    cv.waitKey(1) # для закрытия от клавиатуры, если не Windows

In [4]:
file_image = '../data/images/ayvas_picture.jpg'
image = cv.imread(file_image)
show_my_image(image)

In [5]:
hsv_min = np.array((0, 54, 5), np.uint8)
hsv_max = np.array((187, 255, 253), np.uint8)

fn = file_image # имя файла, который будем анализировать
img = cv.imread(fn)

hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV ) # меняем цветовую модель с BGR на HSV
thresh = cv.inRange(hsv, hsv_min, hsv_max ) # применяем цветовой фильтр

contours0, hierarchy = cv.findContours(thresh.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# перебираем все найденные контуры в цикле
for cnt in contours0:
    rect = cv.minAreaRect(cnt) # пытаемся вписать прямоугольник
    box = cv.boxPoints(rect) # поиск четырех вершин прямоугольника
    box = np.int0(box) # округление координат
    cv.drawContours(img,[box],0,(255,0,0),2) # рисуем прямоугольник

cv.imshow('contours', img) # вывод обработанного кадра в окно

cv.waitKey()
cv.destroyAllWindows()
cv.waitKey(1)

-1

# Как можно улучшить точность?

Сделаем небольшую модификацию. Будем фильтровать результаты по площади

In [6]:
box = np.int0(box) # округление координат
area = int(rect[1][0]*rect[1][1]) # вычисление площади
if area > 900:
    cv.drawContours(img,[box],0,(255,0,0),2)

In [7]:
hsv_min = np.array((0, 54, 5), np.uint8)
hsv_max = np.array((187, 255, 253), np.uint8)


fn = file_image # имя файла, который будем анализировать
img = cv.imread(fn)

hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV) # меняем цветовую модель с BGR на HSV
thresh = cv.inRange(hsv, hsv_min, hsv_max) # применяем цветовой фильтр

show_my_image(thresh)

In [8]:
contours0, hierarchy = cv.findContours(thresh.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# перебираем все найденные контуры в цикле
for cnt in contours0:
    rect = cv.minAreaRect(cnt) # пытаемся вписать прямоугольник
    box = cv.boxPoints(rect) # поиск четырех вершин прямоугольника
    box = np.int0(box) # округление координат
    area = int(rect[1][0]*rect[1][1]) # вычисление площади
    if area > 75000 and area < 500000:
        print(area)
        cv.drawContours(img,[box],0,(255,0,0),2)

cv.imshow('contours', img) # вывод обработанного кадра в окно

cv.waitKey()
cv.destroyAllWindows()
cv.waitKey(1)

226188


-1

# Домашнее задание 

## Задание № 1. Считаем круги

In [9]:
file_image = '../data/images/bubbles_2.png'
image = cv.imread(file_image)
show_my_image(image)

* Посчитать количество кругов на картинке с помощью OpenCV

In [10]:
output = image.copy() # Создаем копию исходного изображения на всякий случай
# cv2.HoughCircles Требует  8-bit-й одно-канальное изображение, 
# поэтому конвертируем RGB цветное изображени в серые оттенки.
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 

# обнаруживаем круги на изображении
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1.20, minDist=12, param1=30, param2=30, minRadius=12, maxRadius=22)

if circles is not None:  # Проверяем,что нашлись хотя бы какие-то круги
    # преобразовываем координаты (x, y) и радиус кругов в целые числа
    # обрабатывает преобразование наших кругов из координат с плавающей точкой (x, y) в целые числа, 
    # это позволит нам рисовать их на нашем выходном изображении.
    
    circles = np.round(circles[0, :]).astype('int') # 

    print('Найдено: ',len(circles), 'кругов')
    for (x, y, r) in circles:  # Пробегаем/итеррируемся по всем координатам и радиусам кругов
        
        
        # рисуем круг на выходном изображении, затем рисуем прямоугольник
        # в соответствии с центром круга
        cv.circle(output, (x, y), r, (0, 255, 0), 4)
        cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)

    show_my_image(output)
    
else: print('Кругов нет')

Найдено:  255 кругов


## Задание № 2. Развешиваем картины

* Выделить все картины на фотографии
* Посчитать площадь картин. 
* Подписать площадь каждой картины на фотографии

In [11]:
file_image = '../data/images/pictures_2.jpg'

image = cv.imread(file_image)  # file_image - имя файла, который будем анализировать
show_my_image(image)

In [12]:
hsv_min = np.array((65, 65, 65), np.uint8)
hsv_max = np.array((255, 255, 255), np.uint8)

fn = file_image # имя файла, который будем анализировать
img = cv.imread(fn)
ret, threshold_image = cv.threshold(img, 0, 255, cv.THRESH_TOZERO)
thresh = cv.inRange(threshold_image, hsv_min, hsv_max) # применяем цветовой фильтр
contours0, hierarchy = cv.findContours(thresh.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# перебираем все найденные контуры в цикле
for cnt in contours0:
    rect = cv.minAreaRect(cnt) # пытаемся вписать прямоугольник
    box = cv.boxPoints(rect) # поиск четырех вершин прямоугольника
    box = np.int0(box) # округление координат
    area = int(rect[1][0]*rect[1][1]) # вычисление площади
    if area > 3000 and area < 18000:
        cv.putText(img,
                   str(area),
                   (int(cv.moments(cnt)['m10']/cv.moments(cnt)['m00']-20),
                    int(cv.moments(cnt)['m01']/cv.moments(cnt)['m00'])),
                   cv.FONT_HERSHEY_SIMPLEX,
                   0.4,
                   (0, 0, 255),
                   1)
        cv.drawContours(img,
                        [box],
                        0,
                        (0, 0, 255),
                        5) # рисуем прямоугольник

show_my_image(img)

In [13]:
show_my_image(thresh)

# Дополнительно

* Детальный разбор детекции объектов с помощью цветовой сегментации: 

https://towardsdatascience.com/object-detection-via-color-based-image-segmentation-using-python-e9b7c72f0e11

* Статья по openCV на русском https://tproger.ru/translations/opencv-python-guide/